<h1>Home Assignment injunctions!</h1>
<li>No Loops. Use only elementwise operations</li>
<li>Your code should work for any data in the provided format. Data values (example, customer ids, product ids, number of customers, etc.) should NEVER appear in your code!</li>

<h1>Problem 1:</h1>
Write a function that reads timeseries pricing data from a file into a pandas dataframe and then groups the data as follows:
<li>The arguments to the function are the filename and a threshold number
<li>The function reads the data in the file and creates a new column "pct_change" with the one day percent change
<li>Then groups the data into four categories:
<ul>
<li>"High+" if the percent change is greater than the threshold 
<li>"Low+" if the percent change is zero or positive and less than or equal to the threshold% 
<li>"Low-" if the percent change is negative but greater than or equal to -1 * the threshold
<li>"High-" if the percent change is less than -1 * the threshold
</ul>
<li>The function should return a dataframe that contains three columns (count, mean, stdev) and four index values (High+, High-, Low+, Low-)
    <p><b>Note: </b>we have to deal with nan percent changes. Make sure that you don't count a NaN in any of the four categories! (see https://pandas.pydata.org/docs/reference/api/pandas.isna.html)  <p>
For the sample data your function should return a dataframe with the following values for a threshold of 1.0:

<pre>
        count	mean	stdev
High+	63	   2.201357	1.403010
High-	58	  -2.227977	1.133905
Low+	70	   0.466946	0.292076
Low-	60	  -0.506772	0.278789

</pre>

    

<h3>read_csv</h3>
The pandas <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">read_csv</a> function reads data from a delimited file into a pandas dataframe.



<h2>Sample data</h2>
Use the attached AAPL.csv file

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

def change_groups(datafile,threshold=1):
    # read into pandas data frame
    df = pd.read_csv(datafile)

    # calc the pct change
    df['pct_change'] = df['Adj Close'].pct_change(1)*100
    
    # assign values 
    df['category'] = df.apply(lambda x: 
                            np.nan if np.isnan(x['pct_change']) else # manage the situation where nan values appear
                            'High+' if x['pct_change'] > threshold 
                            else 'Low+' if x['pct_change'] >= 0 and x['pct_change'] <= threshold
                            else 'Low-' if x['pct_change'] >= -1*threshold and x['pct_change'] < 0
                            else 'High-',axis=1)
    # group by assigned categories and then apply the aggregate functions
    groups = df.groupby(['category'], as_index=False).agg({'pct_change':['count','mean','std']})
    return groups
change_groups('AAPL.csv',1)


category pct_change                    
                count      mean       std
0    High+         63  2.201357  1.403010
1    High-         58 -2.227977  1.133905
2     Low+         70  0.466946  0.292076
3     Low-         60 -0.506772  0.278789

<h1>Problem 2</h1>
A manufacturer has data on orders from customers and product prices in two dataframes (see below). They want to use this data to answer the following questions:
<ol>
    <li>Which customer is responsible for the most revenue</li>
    <li>Which customer is responsible for the highest profit</li>
    <li>Which product is responsible for the highest (dollar) profit</li>
    <li>Which customer and product combination is responsible for the most orders</li>
</ol>
<p>
Obviously, your code should work for any actual data values and pandas dataframes of any length!
<p>For the data below: your answers should be:
    
<pre>
Customer with most profit: 005
Customer with most revenue: 007
Product with most profit: 011
Customer 001 with product 010 with 4 orders is the most ordered customer product pair
</pre>


<h2>Useful functions:</h2>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html">pd.sort_values</a> </li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html">pandas dataframe join</a> the last example on the linked page is probably what you need here!</li>
<li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html">pd.groupby</a></li>
<li><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html">pandas join</a></li>

In [3]:
import pandas as pd
import numpy as np
order_data = pd.DataFrame([["001","010",24],
                           ["007","012",35],
                           ["001","011",15],
                           ["005","010",30],
                           ["007","011",17],
                           ["005","011",81],
                           ["001","010",32],
                           ["007","012",89],
                           ["001","010",16],
                           ["001","010",33]],columns=["customer","product","amount"])
products = pd.DataFrame([['010',22.3,17.2],
                        ['011',11.7,5.5],
                        ['012',62.5,61.4]],columns=['product','price','cost'])                     

In [4]:
order_data

,customer,product,amount
0,001,010,24
1,007,012,35
2,001,011,15
3,005,010,30
4,007,011,17
5,005,011,81
6,001,010,32
7,007,012,89
8,001,010,16
9,001,010,33


In [5]:
products

,product,price,cost
0,010,22.3,17.2
1,011,11.7,5.5
2,012,62.5,61.4


In [6]:
# if we force both data tables to have the same set of row names then pandas will allign like-rows with each other
all_data = order_data.set_index('product').join(products.set_index('product'))
all_data['revenue'] = all_data['price']*all_data['amount'] # revenue=price(product_id)*amount
all_data['profit'] = all_data['amount']*(all_data['price']-all_data['cost'])  # profit = selling price - cost 
all_data


,customer,amount,price,cost,revenue,profit
product,,,,,,
010,001,24,22.3,17.2,535.2,122.4
010,005,30,22.3,17.2,669.0,153.0
010,001,32,22.3,17.2,713.6,163.2
010,001,16,22.3,17.2,356.8,81.6
010,001,33,22.3,17.2,735.9,168.3
011,001,15,11.7,5.5,175.5,93.0
011,007,17,11.7,5.5,198.9,105.4
011,005,81,11.7,5.5,947.7,502.2
012,007,35,62.5,61.4,2187.5,38.5


In [18]:
# Q1: Which customer is responsible for the highest profit

# group by customer and then by profit
q1 = all_data.groupby('customer')
q1_groups = q1['profit'].sum()
print(q1_groups)
print("The customer that produced the most profit: {}".format(q1_groups.idxmax()))



customer
001    628.5
005    655.2
007    241.8
Name: profit, dtype: float64
The customer that produced the most profit: 005


In [19]:
# Q2: Which customer is responsible for the most revenue

# group by customer and then revenue 

q2 = all_data.groupby('customer')
q2_groups = q1['revenue'].sum()
print(q2_groups)
print("The customer that produced the most revenue: {}".format(q2_groups.idxmax())) # idxmax gets the max index in the aggregated


customer
001    2517.0
005    1616.7
007    7948.9
Name: revenue, dtype: float64
The customer that produced the most revenue: 007


In [13]:
# Q3: Which product is responsible for the highest (dollar) profit

# group by product and then sum 
q3 = all_data.groupby('product')
q3_groups = q3['profit'].sum()
print(q3_groups)
print("The product that produced the highest profit: {}".format(q3_groups.idxmax()))

product
010    688.5
011    700.6
012    136.4
Name: profit, dtype: float64
The product that produced the highest profit: 011


In [16]:
# Q4: Which customer and product combination is responsible for the most orders

# group by customer, then by product, and then sum 
q4 = all_data.groupby(['customer','product']).size()
print(q4)
print("Customer {} with product {} with {} orders is the most ordered customer product pair".format(q4.idxmax()[0],q4.idxmax()[1],max(q4)))

customer  product
001       010        4
          011        1
005       010        1
          011        1
007       011        1
          012        2
dtype: int64
Customer 001 with product 010 with 4 orders is the most ordered customer product pair


<h1>Problem 3</h1>
In this problem you'll get some practice getting and combining data from the St. Louis Federal Reserve (FRED). Get the following data from FRED (01/01/2010 to 12/31/2022):

<pre>
"TB3MS" #3 month t-bill market yield 
"DGS10" #10 year constant maturity government bond market yield
"NB000334Q" #Real GDP index quarterly (index = 100 at 2012)
"CPIAUCSL" #Consumer price index for all urban consumers seasonally adjusted
</pre>

You will need to register with the federal reserve and get an API key. 

1. Create a new account at https://fredaccount.stlouisfed.org/login/secure/
2. Go to My Account (top right corner) once you're logged in and select the Api-key option. Copy the API key and save it in a variable in your notebook

Since these data items have different frequencies (some are daily, some monthly, some quarterly), make separate fred api calls for each. 

Then, as a proxy for the stock market, get data for the ticker SPY,the S&P ETF, from yfinance. 

Align all the data to the end of the business quarter (i.e., use the value on the last day of each quarter).

For the ETF, calculate one day percent changes and shift the data back by one quarter (we're interested in the correlation between macroeconomic data in one quarter and the performance of the S&P in the next quarter). For example, if the quarterly percentage change on 3/31 is 5% and on 6/30 is 2.5% (i.e., the change in value between 3/31 and 6/30), we want to align the percent change on 6/30 with the the macroeconomic data as of 3/31. So we need to replace the data on 3/31 by 2.5%

Using the pandas join function, join all the data into one dataframe with the quarter end date as the index

Generate the correlation matrix. This is what you should get:

<pre>
            TB3MS	     DGS10	    NB000334Q	 CPIAUCSL	   SPY
TB3MS   	1.000000	0.409817	0.063098	0.049515	-0.079082
DGS10	    0.409817	1.000000	0.056068	0.060827	-0.329453
NB000334Q	0.063098	0.056068	1.000000	0.162265	-0.009601
CPIAUCSL	0.049515	0.060827	0.162265	1.000000	-0.398615
SPY	        -0.079082	-0.329453	-0.009601	-0.398615	1.000000

</pre>

<h3>Notes:</h3>

1. positive numbers will shift forward while negative numbers will shift backward

    
2. To rename a column, use df.rename(columns={"old_name":"new_name"})

3. To join two dataframes use:
    new_df = old_df1.join(old_df2)

<h3>Install fredapi</h3>

In [4]:
!pip install fredapi

<h3>Using the api</h3>
<li>Make sure you have the api key</li>
<li>Then use the example below to get data</li>

In [8]:
API_KEY = '7520fc0658bb92fdbb6e492c47dbfcf9'
# with open("../../credentials/fred") as f:
#     API_KEY = f.read().strip()

In [20]:
#Import fredapi and datetime
import fredapi as fa
import datetime as datetime

#Copy your key into 
#Create a fred object that knows the api key
#Each request will then automatically contain that key
fred = fa.Fred(api_key=API_KEY)

#Get a data series
#For example, to get the TB3MS series use
start = datetime.datetime(month=1,day=1,year=2010)
end=datetime.datetime(month=12,day=31,year=2022)

fred_data_tb3ms = fred.get_series("TB3MS",observation_start=start,observation_end=end).resample('BQ').last().reset_index(name='tb3')
fred_data_dgs = fred.get_series("DGS10",observation_start=start,observation_end=end).resample('BQ').last().reset_index(name='dgs')
fred_data_nb = fred.get_series("NB000334Q",observation_start=start,observation_end=end).resample('BQ').last().pct_change().reset_index(name='gdp')
fred_data_cpi = fred.get_series("CPIAUCSL",observation_start=start,observation_end=end).resample('BQ').last().pct_change().reset_index(name='cpi')

# make the data frame 
processed_fred_data = fred_data_cpi.merge(fred_data_nb).merge(fred_data_dgs).merge(fred_data_tb3ms)



<h3>Problem setup</h3>

In [15]:
#import yfinance as np
!pip install yfinance --upgrade
#np.__version__

     |████████████████████████████████| 65 kB 6.4 MB/s eta 0:00:011
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached lxml-4.9.3.tar.gz (3.6 MB)
  Using cached frozendict-2.3.8-cp39-cp39-macosx_10_9_x86_64.whl (36 kB)
     |████████████████████████████████| 928 kB 8.5 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for lxml: filename=lxml-4.9.3-cp39-cp39-macosx_10_9_x86_64.whl size=1560700 sha256=a26b41d801401664037591392c1e82faf606f8cf396392a571dce2af08a18059
  Stored in directory: /Users/joshuagrajales/Library/Caches/pip/wheels/5c/05/aa/530f84480d476c5bb9ea09877eea78fb144ec047fbb00ee2ca
  Created wheel for peewee: filename=peewee-3.16.3-cp39-cp39-macosx_10_9_x86_64.whl size=267751 sha256=e3033e0557bd514c2666596f7b9976e9d04b48cf835fd37c5bed950cea8d22bd
  Stored in directory: /Users/joshuagrajales/Library/Caches/pip/wheels/46/c8/76/da8e4e28595d45fac70db7

In [21]:
import fredapi as fa
fred = fa.Fred(api_key=API_KEY)

#Date range
import datetime
import numpy as np
start = datetime.datetime(month=1,day=1,year=2010)
end=datetime.datetime(month=12,day=31,year=2022)

#DATA IDENTIFIERS
tbill3 = "TB3MS" #3 month t-bill market yield 
tnote10 = "DGS10" #10 year constant maturity market yield
gdp_index = "NB000334Q" #Real GDP index quarterly (index = 100 at 2012)
cpi = "CPIAUCSL" #Consumer price for all urban consumers seasonally adjusted

import yfinance as yf
spy_df = yf.download('SPY',start,end)
spy_df.info()

[*********************100%%**********************]  1 of 1 completed
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3272 entries, 2010-01-04 to 2022-12-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3272 non-null   float64
 1   High       3272 non-null   float64
 2   Low        3272 non-null   float64
 3   Close      3272 non-null   float64
 4   Adj Close  3272 non-null   float64
 5   Volume     3272 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 178.9 KB


In [22]:
spy_data = spy_df.resample('BQ').last().pct_change().shift(-1) # toggle between -1 and 1 to determine whats correct
processed_spy_data = spy_data['Adj Close'].reset_index(name='spy')
processed_spy_data


,Date,spy
0,2010-03-31,-0.113580
1,2010-06-30,0.111616
2,2010-09-30,0.107608
3,2010-12-31,0.058974
4,2011-03-31,0.000259
5,2011-06-30,-0.138172
6,2011-09-30,0.116182
7,2011-12-30,0.126909
8,2012-03-30,-0.028441
9,2012-06-29,0.063472


In [28]:
corr_matrix = processed_fred_data.join(processed_spy_data).corr()
corr_matrix.drop(['Date','index'],axis=1).drop(['index','Date'],axis=0)

,cpi,gdp,dgs,tb3,spy
cpi,1.000000,0.162265,0.060827,0.049515,-0.398616
gdp,0.162265,1.000000,0.056068,0.063098,-0.009602
dgs,0.060827,0.056068,1.000000,0.409817,-0.329453
tb3,0.049515,0.063098,0.409817,1.000000,-0.079082
spy,-0.398616,-0.009602,-0.329453,-0.079082,1.000000
